In [1]:
!arduino-cli compile --fqbn arduino:avr:uno ~/Documents/Projects/Arduino/pwm_pr


Sketch uses 1150 bytes (3%) of program storage space. Maximum is 32256 bytes.
Global variables use 9 bytes (0%) of dynamic memory, leaving 2039 bytes for local variables. Maximum is 2048 bytes.

Used platform Version Path
arduino:avr   1.8.6   /home/yehori/.arduino15/packages/arduino/hardware/avr/1.8.6


In [4]:
PROJECT = "empty_sketch"

!arduino-cli compile --fqbn arduino:avr:uno ~/Documents/Projects/Arduino/"{PROJECT}"
!arduino-cli upload -p /dev/ttyUSB0 --fqbn arduino:avr:uno ~/Documents/Projects/Arduino/"{PROJECT}"


Sketch uses 444 bytes (1%) of program storage space. Maximum is 32256 bytes.
Global variables use 9 bytes (0%) of dynamic memory, leaving 2039 bytes for local variables. Maximum is 2048 bytes.

Used platform Version Path
arduino:avr   1.8.6   /home/yehori/.arduino15/packages/arduino/hardware/avr/1.8.6
New upload port: /dev/ttyUSB0 (serial)


In [ ]:
import os
from IPython.display import display, clear_output
from ipywidgets import widgets, Layout

ARDUINO_DIR = os.path.expanduser("~/Documents/Projects/Arduino/arduino_programs")

def list_projects():
    return [d for d in os.listdir(ARDUINO_DIR) if os.path.isdir(os.path.join(ARDUINO_DIR, d))]

def compile_project(project):
    cmd = f"arduino-cli compile --fqbn arduino:avr:uno {os.path.join(ARDUINO_DIR, project)}"
    print(f"Compiling {project}...")
    result = os.system(cmd)
    if result == 0:
        print("Compilation successful!")
    else:
        print("Compilation failed.")
    return result == 0

def upload_project(project):
    cmd = f"arduino-cli upload -p /dev/ttyUSB0 --fqbn arduino:avr:uno {os.path.join(ARDUINO_DIR, project)}"
    print(f"Uploading {project}...")
    result = os.system(cmd)
    if result == 0:
        print("Upload successful!")
    else:
        print("Upload failed.")
    return result == 0

def on_project_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output(wait=True)
            print(f"Selected project: {change['new']}")

def on_compile_click(b):
    with output:
        compile_project(dropdown.value)

def on_upload_click(b):
    with output:
        upload_project(dropdown.value)

def on_run_click(b):
    with output:
        if compile_project(dropdown.value):
            upload_project(dropdown.value)

# Create widgets
projects = list_projects()
dropdown = widgets.Dropdown(
    options=projects,
    description='Project:',
    disabled=False,
    layout=Layout(width='300px')
)

compile_button = widgets.Button(description="Compile")
upload_button = widgets.Button(description="Upload")
run_button = widgets.Button(description="Run")

# Create output widget
output = widgets.Output()

# Attach callbacks
dropdown.observe(on_project_change, names='value')
compile_button.on_click(on_compile_click)
upload_button.on_click(on_upload_click)
run_button.on_click(on_run_click)

# Create a horizontal box for buttons
button_box = widgets.HBox([compile_button, upload_button, run_button])

# Display widgets
display(dropdown)
display(button_box)
display(output)

# Initial project display
with output:
    print(f"Selected project: {dropdown.value}")


Dropdown(description='Project:', layout=Layout(width='300px'), options=('.git', 'traffic_l', 'sketch_oct8a', '…

Output()

In [ ]:
import serial
import time
from IPython.display import display
import ipywidgets as widgets
import threading
import statistics
from collections import deque

class ArduinoMonitor:
    def __init__(self, port='/dev/ttyUSB0', baudrate=9600, max_lines=10, avg_lines=5):
        self.port = port
        self.baudrate = baudrate
        self.max_lines = max_lines
        self.avg_lines = avg_lines
        self.is_running = False
        self.lines = deque(maxlen=max_lines)
        self.rolling_avg_data = deque(maxlen=avg_lines)
        
        self.output = widgets.HTML()
        self.stop_button = widgets.Button(description="Stop Monitoring")
        self.stop_button.on_click(self.stop_monitoring)
        self.rolling_avg_checkbox = widgets.Checkbox(description=f"Enable Rolling Average (last {self.avg_lines})", value=False)
        
        self.control_box = widgets.HBox([self.stop_button, self.rolling_avg_checkbox])
        self.main_box = widgets.VBox([self.control_box, self.output])
        
    def start_monitoring(self):
        self.is_running = True
        self.ser = serial.Serial(self.port, self.baudrate, timeout=1)
        self._update_output(f"Listening to Arduino on {self.port}")
        threading.Thread(target=self._monitor_thread, daemon=True).start()
        
    def stop_monitoring(self, _=None):
        self.is_running = False
        
    def _monitor_thread(self):
        while self.is_running:
            if self.ser.in_waiting:
                line = self.ser.readline().decode('utf-8').strip()
                if line:
                    try:
                        value = float(line)
                        self.lines.append(value)
                        self.rolling_avg_data.append(value)
                        self._update_output()
                    except ValueError:
                        self._update_output(f"Non-numeric data received: {line}")
            time.sleep(0.1)
        
        self.ser.close()
        self._update_output("Serial connection closed")
        
    def _update_output(self, status_message=None):
        output_lines = []
        if status_message:
            output_lines.append(f"<b>{status_message}</b>")
        
        if self.rolling_avg_checkbox.value and self.rolling_avg_data:
            avg = statistics.mean(self.rolling_avg_data)
            output_lines.append(f"<b>Rolling Average (last {self.avg_lines} readings): {avg:.2f}</b>")
        
        output_lines.append(f"<b>Last {self.max_lines} readings:</b>")
        for i, l in enumerate(self.lines, 1):
            output_lines.append(f"{i}. {l}")
        
        self.output.value = "<br>".join(output_lines)
    
    def display(self):
        display(self.main_box)
        self.start_monitoring()

# Usage
# Default: monitor = ArduinoMonitor()
monitor = ArduinoMonitor('/dev/ttyUSB0', 9600, max_lines=15, avg_lines=120)
monitor.display()
